In [19]:
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize

import numpy as np
import pandas as pd
import csv
import math
from collections import Counter
import matplotlib.pyplot as plt

s=pd.read_csv('C:/Users/geun/KoreanNewsChronicle/Topic_Abstract/Data/201905.csv',encoding='utf-8')

# df = df.dropna()

# # 불용어 제거
# stopword = ['A씨', '만', '억', '조', '차', '그', '시', '대', '위', '분', '측', '곳']

# for index, row in df.iterrows():
#     topic = row['topics'].split(', ')
#     for str in topic:
#         if(str in stopword):
#             topic.remove(str)

#     topic_str = ", ".join(topic)
#     df.loc[index, 'topics'] = topic_str

# print('원래 기사 수 : ',len(df))
# # 결측지 행 제거 후 topics에 있는 것을 추출
# dp = df.duplicated(['headline'], keep='first')
# count = 0
# for b in dp:            #중복된 기사 제거
#     if(b == True):
#         df.drop(df.index[count], inplace = True)
#         count -= 1
#     count += 1

# print('중복된 기사 제거후 기사수 : ',len(df))

# tmp_index=[]

# for index, row in df.iterrows():
#     tmp = row['topics'].split(',')
#     if len(tmp) < 4:
#         tmp_index.append(index)

# s = df.drop(tmp_index)

In [20]:

news_topic = s['topics'].tolist()

# # CountVectrizer로 토큰화
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(news_topic)

X = normalize(X)

neigh = NearestNeighbors(n_neighbors=2)
nbrs = neigh.fit(X)
distances, indices = nbrs.kneighbors(X)


m = DBSCAN(eps=0.6, min_samples = 5)

m.fit(X)

labels = m.labels_

In [21]:
s['labels'] = labels

hot_topic_index = Counter(s['labels']).most_common()
hot_topic = dict(hot_topic_index)

print(hot_topic)

{-1: 22965, 2: 257, 5: 126, 17: 19, 38: 15, 18: 13, 16: 12, 31: 11, 12: 10, 14: 9, 36: 9, 40: 9, 25: 8, 9: 7, 10: 7, 22: 7, 32: 7, 33: 7, 37: 7, 19: 6, 8: 6, 13: 6, 39: 6, 28: 6, 43: 6, 0: 5, 1: 5, 3: 5, 4: 5, 15: 5, 6: 5, 11: 5, 7: 5, 20: 5, 21: 5, 24: 5, 27: 5, 26: 5, 29: 5, 30: 5, 46: 5, 34: 5, 41: 5, 35: 5, 42: 5, 44: 5, 45: 5, 23: 4}


In [22]:
count = []
for index, row in s['labels'].iteritems():
    if row == -1:
        s = s.drop(index)
    else:
        count.append(hot_topic[row])
        s['labels'][index] = "05,"+str(row)

s['cnt'] = count
s.sort_values(by='labels', ascending=False).to_csv('./Data/201905_cluster_DBSCAN.csv', index=False, header=True,encoding="utf-8-sig")